# Hash

О хэше можно думать как о функции, отображающей произвольный объект в целое число:

$$
  h\colon \mbox{"какой-то объект"} \mapsto \mathbb Z
$$

"Дефолтное" вычисление хэша в Питоне

In [1]:
hash("Hello world!")

4840737576135146834

Хэш-функция (хорошая) должна обладать несколькими свойствами.

Так, хэш от объекта не должен меняться со временем (в рамках текущей сессии работы программы):

In [2]:
hash("Hello world!")

4840737576135146834

Ещё при даже небольшом изменении объекта — хэш должен меняться сильно (чтобы нельзя было по значению хэша что-то сказать об объекте):

In [3]:
hash("Hello world?")

-1706146803320632315

## Hash Under Dict's Hood

Питонский словарь $d$ — отображает ключи $k$ в значения $v$:

$$
  d\colon k \mapsto v
$$

Но под капотом в словаре сидит хэш таблица: по ключам сначала считаются хэши $h$:

$$
  d\colon k \mapsto h \mapsto v
$$

In [4]:
d = {
    'Jane': 'Porter',
    'Kitty': 'Softpaws',
}

# В одном и том же словаре Питона также допускается наличие ключей "разной природы".
# Например, строковые, числовые.
# (но использовать разные типы ключей в одном словаре — это не очень best practice)

In [5]:
d

{'Jane': 'Porter', 'Kitty': 'Softpaws'}

Изменим ("испортим") способ вычисления хэша в Питоне, сделав хэш одинаковым для всех объектов:

In [6]:
# На всякий случай сохраним нормальную хэш-функцию:
old_adequate_hash = hash

def hash(o):
    return 1

In [7]:
hash("Hello world!")

1

In [8]:
hash(123)

1

In [9]:
old_adequate_hash("Hello world!")

4840737576135146834

И попробуем ещё раз создать словарь.
Теперь будут сплошные коллизии.
Что получится в итоге?

In [10]:
d = {
    'Jane': 'Porter',
    'Kitty': 'Softpaws',
}

In [11]:
d

{'Jane': 'Porter', 'Kitty': 'Softpaws'}

Словарь получился нормальный (в нём есть все ключи), потому что при записи в хэш таблицу Питон смотрит не только на хэш (который в данном случае был одинаковый для всех ключей, то есть коллизия), но и на сами ключи: если ключи отличаются, то каждый сохраняется отдельно (Питон умеет как-то разрешать коллизии).

Если же попытаться записать в словарь значения под ключами, которые Питон считает равными:

In [12]:
d[0] = "Hello"
d[0.0] = "world!"

In [13]:
d

{'Jane': 'Porter', 'Kitty': 'Softpaws', 0: 'world!'}

...то последняя запись перепишет первую, потому что в этом случае происходит не просто коллизия (совпадение хэшей) — Питон считает, что это буквально один и тот же ключ (так как ещё и сами ключи равны), поэтому обновляет существующую запись.

In [14]:
0 == 0.0

True

## Hash Tables' Pros: Fast Search

Хэши под капотом словаря обеспечивают более быстрый поиск в нём (по сравнению, например, со списком).

Посмотрим на время работы поиска в списке (тупо просмотр элементов от первого до последнего, пока не найдём искомый или пока список просто не закончится):

In [15]:
BIG_NUMBER = 1000001

In [16]:
l = list(range(1, BIG_NUMBER))

In [17]:
%%timeit

# Поиск такого, которого нет

-1 in l

5.18 ms ± 40.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit

# Поиск стоящего в конце

(BIG_NUMBER - 1) in l

5.72 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit

# Поиск стоящего в начале

1 in l

23.7 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Теперь создадим словарь с таким же списком ключей (и какими-то значениями).
(При этом для чистоты эксперимента не забудем "откатиться" к нормальной хэш-функции!)

In [20]:
hash = old_adequate_hash

d = {
    k: k for k in range(1, BIG_NUMBER)
}

In [21]:
%%timeit

# Нет среди ключей

-1 in d

29.2 ns ± 0.196 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [22]:
%%timeit

# Последний из добавленных ключей

(BIG_NUMBER - 1) in d

65.4 ns ± 0.784 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [23]:
%%timeit

# Первый занесённый ключ

1 in d

29 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


(Именно "первый занесённый" и "последний занесённый" ключи, а не "первый" и "последний" — потому что в словарях вообще нет "порядка" ключей, словарь — это просто отображение ключей в значения.)

Снова испортим хэш-функцию, сделав много коллизий.
Замедлит ли это время работы поиска в словаре?
Ведь если Питон создаёт цепочки для каждого хэша, то поиск при сплошных коллизиях выродится просто в поиск в списке...

In [24]:
def hash(o):
    return 1

In [25]:
d = {
    k: k for k in range(1, BIG_NUMBER)
}

In [26]:
%%timeit

(BIG_NUMBER - 1) in d

64.5 ns ± 0.292 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Почему-то всё равно получилось быстро...

Дело в том, что Питон для разрешения коллизий использует подход Open Addressing (он же Closed Hashing).
При котором в случае коллизии наступает поиск другого свободного места в таблице.
Более того, поиск этого свободного места Питон проводит в некотором роде *случайно* (https://stackoverflow.com/a/9022664/8094251).
Таким образом, даже "полная деградация" хэш-функции не позволила испортить словарь.

Подход к разрешению коллизий через цепочки для каждого хэша называется Separate Chaining (он же Open Hashing).

Про лингвистические нюансы в именованиях способов разрешения коллизий (когда и чего там открыто, закрыто) рекомендуется посмотреть заметку: https://stackoverflow.com/a/9124535/8094251.

## Hash-Brother of Dict

Кроме словарей, хэши используются во множествах (неупорядоченных коллекциях уникальных элементов):

In [27]:
{1, 2, 3, 1, 1, 1}

{1, 2, 3}

Уникальность как раз определяется по хэшу (точнее, сначала по хэшу, а в случае коллизии — по значению).

## "Total Collision"

Смоделируем ситуацию, когда во множество добавляется два элемента с одинаковыми хэшами, и которые ещё одинаковые в смысле равенства (но при этом всё-таки разные 🙂).

In [28]:
def hash(o):
    return 1

In [29]:
class A:                         # Класс — шаблон
    def __init__(self, number):  # Конструктор — как проинициализировать создаваемый объект self
        self._number = number    # "Привяжем" к объекту число

    def __eq__(self, other):     # Как сравнивается текущий объект self и какой-то другой other через == (то есть "self == other")
        return True              # Считаем, что вообще все объекты класса A равны
    
    def __hash__(self):          # Как считать хэш от объекта
        return hash(self)        # Делегируем вычисление хэша дефолтной (сломанной в этом ноутбуке) функции hash

In [30]:
a = A(1809)
b = A(2024)

In [31]:
a._number

1809

In [32]:
b._number

2024

In [33]:
a == b

True

In [34]:
s = {a, b}

Получили множество из одного объекта, хотя клали в него два.
(Просто они для Питона оказались полностью одинаковыми.)

In [35]:
len(s)

1

In [36]:
s

{<__main__.A at 0x7f5ae5737a00>}

In [37]:
a, b

(<__main__.A at 0x7f5ae5737a00>, <__main__.A at 0x7f5ae5734d00>)

Видно (по крокозябрному выводу выше), что во множестве остался тот объект, который клали в него первым (второй оказался "таким же", поэтому он просто "пролетел").

## Appendix: Fast and Slow Remove from List

Создадим большой список и посмотрим, сколько времени занимает многократное удаление из конца и из начала.

In [38]:
l = list(range(BIG_NUMBER))

In [39]:
%%time

for i in range(BIG_NUMBER):
    l.pop()

CPU times: user 74.3 ms, sys: 35 µs, total: 74.3 ms
Wall time: 74 ms


Удалять из конца — быстро.

In [40]:
l = list(range(BIG_NUMBER))

In [41]:
%%time

for i in range(10**5):
    l.pop(0)

CPU times: user 33.3 s, sys: 11.8 ms, total: 33.3 s
Wall time: 33.3 s


А из начала — медленно.
Потому что удаление из начала — это не просто удаление, но ещё и перестройка списка (удаление = построение нового списка без удаляемого элемента).